In [1]:
#libraries used

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from scipy import stats
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
import os
from statistics import mean
import eppy as eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
from eppy.runner.run_functions import runIDFs
from eppy.results import readhtml
import random



In [2]:
os.chdir('C:\\Users\\amitc_crl\\OneDrive\\Documents\\GitHub\\IIT-PhD\\CAE526\\P1')

df = pd.read_csv('CAE526-P1_HH UtilityData_Mitchell.csv')

gsf = 111135

df2012 = df.iloc[7:19]
df2013 = df.iloc[19:31]
df2020 = df.iloc[37:49]
eui2012 = (df2012.sum(axis=0)[1] + df2012.sum(axis=0)[3] + df2012.sum(axis=0)[5])/gsf
eui2013 = (df2013.sum(axis=0)[1] + df2013.sum(axis=0)[3] + df2013.sum(axis=0)[5])/gsf
eui2020 = (df2020.sum(axis=0)[1] + df2020.sum(axis=0)[3] + df2020.sum(axis=0)[5])/gsf

print('EUI 2012 = ' + str(eui2012))
# print('EUI 2013 = ' + str(eui2013))
# print('EUI 2020 = ' + str(eui2020))

df2012 = df2012.reset_index()

df2012.head()

EUI 2012 = 102.36085121698834


,index,MONTH,STEAM [kBTU],ELEC [kWh],ELEC [kBTU],CHW [Ton-hr],CHW [kBTU],DHW [gal],AVG TEMP [F]
0,7,Jan-12,1836810.0,99010.0,337921.1,0.0,0.0,152900.0,31.3
1,8,Feb-12,1677000.0,95168.0,324808.4,0.0,0.0,142700.0,33.3
2,9,Mar-12,776900.0,89106.0,304118.8,0.0,0.0,158100.0,53.6
3,10,Apr-12,743900.0,87680.0,299251.8,0.0,0.0,171200.0,50.9
4,11,May-12,116300.0,80520.0,274814.8,3775.0,45300.0,157500.0,66.0


In [3]:
print('hi')

hi


In [4]:
def cvrsme(meterData, simData):
    data = []
    n = len(meterData)
    y_bar = mean(meterData)
    for m,s in zip(meterData,simData):
        data.append((((m-s)**2)/(n-1)))
    CVRSME = 100*(1/y_bar)*(sum(data)**(1/2))
    return CVRSME

def nmbe(meterData, simData):
    data = []
    n = len(meterData)
    y_bar = mean(meterData)
    for m,s in zip(meterData,simData):
        data.append(m-s)
    NMBE = (((sum(data))/((n-1)*y_bar))*100)
    return NMBE

In [5]:
simData = pd.read_csv('data.csv')

elecSim = simData['Electricity [kWh]']
elecMet = df2012['ELEC [kWh]']

print('Electricity CVRSME:')
print(cvrsme(elecMet, elecSim))

chwSim = simData['Cooling [kBtu]']
chwMet = df2012['CHW [kBTU]']

print('Chilled Water CVRSME:')
print(cvrsme(chwMet, chwSim))

heatSim = simData['Heating [kbtu]']
heatMet = df2012['STEAM [kBTU]']

print('District Heating CVRSME:')
print(cvrsme(heatMet, heatSim))

Electricity CVRSME:
25.171269287918843
Chilled Water CVRSME:
3201.5733062739764
District Heating CVRSME:
42.51478167230881


In [6]:
print('Electricity NMBE:')
print(nmbe(elecMet, elecSim))

print('Chilled Water NMBE:')
print(nmbe(chwMet, chwSim))

print('District Heating NMBE:')
print(nmbe(heatMet, heatSim))

Electricity NMBE:
22.48977043173896
Chilled Water NMBE:
-2179.6510414792965
District Heating NMBE:
8.519809763516434


In [7]:
os.chdir('C:\\Users\\amitc_crl\\OneDrive\\Documents\\GitHub\\IIT-PhD\\CAE526\\P1')
IDF.setiddname("C:\EnergyPlusV22-1-0\Energy+.idd")
epwfile = "USA_IL_Chicago.Midway.Intl.AP.725340_TMY3.epw"
idf1 = IDF('in.idf')
idf2 = IDF('in.idf')
#Multivariate

results = pd.DataFrame(columns=list('ABCDEFGHIKL'))
results.to_csv('results.csv')
global final
final = []

idf1.saveas('eplusTEMP.idf')
idf2.saveas('in.idf')

def f2(params):
    global final
    u,shgc,leakage,lpd,people = params
    results = pd.read_csv('results.csv')
    
    # window = idf1.idfobjects['Window'][2]
    # window.Height = x[0]
    # window = idf1.idfobjects['Window'][3]
    # window.Height = x[1]

    count = -1
    for i in idf1.idfobjects['WindowMaterial:SimpleGlazingSystem']:
        count += 1
        glz = idf1.idfobjects['WindowMaterial:SimpleGlazingSystem'][count]
        if glz.Name == 'Parametric Glass':
            glz.UFactor = (glz.UFactor * u)
            glz.Solar_Heat_Gain_Coefficient = (glz.Solar_Heat_Gain_Coefficient * shgc)

    count = -1
    for i in idf1.idfobjects['ZoneInfiltration:DesignFlowRate']:
        count += 1
        inf = idf1.idfobjects['ZoneInfiltration:DesignFlowRate'][count]
        inf.Flow_per_Exterior_Surface_Area = (inf.Flow_per_Exterior_Surface_Area * leakage)

    count = -1
    for i in idf1.idfobjects['Lights']:
        count += 1
        light = idf1.idfobjects['Lights'][count]
        light.Watts_per_Zone_Floor_Area = (light.Watts_per_Zone_Floor_Area * lpd)

    count = -1
    for i in idf1.idfobjects['People']:
        count += 1
        ppl = idf1.idfobjects['People'][count]
        ppl.People_per_Floor_Area = (ppl.People_per_Floor_Area * people)

    idf1.saveas('eplusTEMP.idf')

    idf = IDF('eplusTEMP.idf', epwfile)
    idf.run(readvars = True)

    data = pd.read_csv('eplusmtr.csv')
    elecSim = list(data['Electricity:Facility [J](Monthly)'].mul(2.7777e-7))
    chwSim = list(data['DistrictCooling:Facility [J](Monthly)'].mul(9e-7))
    heatSim = list(data['DistrictHeating:Facility [J](Monthly) '].mul(9e-7))
    elecCVRSME = cvrsme(elecMet, elecSim)
    chwCVRSME  = cvrsme(chwMet, chwSim)
    heatCVRSME = cvrsme(heatMet, heatSim)

    elecNMBE = nmbe(elecMet, elecSim)
    chwNMBE = nmbe(chwMet, chwSim)
    heatNMBE = nmbe(heatMet, heatSim)
    print(elecCVRSME)
    print(chwCVRSME)
    print(heatCVRSME)

    newRow = [u,shgc,leakage,lpd,people,elecCVRSME,chwCVRSME,heatCVRSME,elecNMBE,chwNMBE,heatNMBE]
    final.append(newRow)
    print(final)
    # newRow = pd.DataFrame([[u,shgc,leakage,lpd,people,elecCVRSME,chwCVRSME,heatCVRSME,elecCVRSME,chwCVRSME,heatCVRSME]], columns=list('ABCDEFGHIKL'))
    # results = results.concat(newRow, ignore_index=True)
            #    columns=['u','shgc','leakage','lpd','people','elecCVRSME','chwCVRSME','heatCVRSME','elecCVRSME','chwCVRSME','heatCVRSME']),
            #    ignore_index = True)
    # results = pd.concat(results, newRow)
    # filehandle = open(fname, 'r').read()
    # htables = readhtml.titletable(filehandle)

    # zoneSE = htables[99]
    # zoneSEtable = zoneSE[1]
    # rowSE = zoneSEtable[24]
    # SolarGainSE = rowSE[2]

    # zoneTEST = htables[141]
    # zoneTESTtable = zoneTEST[1]
    # rowTEST = zoneTESTtable[24]
    # SolarGainTEST = rowTEST[2]

    # winHeight.append(x[0])
    # winHeight2.append(x[1])
    # dGain.append(abs(gainMultiplier*SolarGainTEST-SolarGainSE))

    # print(abs((gainMultiplier*SolarGainTEST)-SolarGainSE))
    # print(x[0],x[1])
    # plt.scatter(winHeight, dGain)
    # plt.scatter(winHeight2, dGain)
    # plt.xlabel("Window Height [m]")
    # plt.ylabel("dGain [Btu/hr]")
    # plt.legend(loc='lower right')
    # plt.show()
    # idf2.saveas('in.idf')
    results.to_csv('results.csv')
    calibration =  ((abs(elecNMBE)) + (abs(chwNMBE)) + (abs(heatNMBE)) + (abs(elecCVRSME)) + (abs(chwCVRSME)) + (abs(heatCVRSME)))
    return calibration


x0 = [1,1,1,1,1]
bnds = ((0.5,1.5),(0.25,1.3),(0.5,1.5),(0.5,1.5),(0.5,1.5))
# mvRes = minimize(f2, x0, method='Nelder-Mead')
mvRes = minimize(f2, x0, bounds=bnds, method='Nelder-Mead', options={'maxiter':200})
# res = minimize_scalar(f2, x0, bounds=(0.1, 2), method='bounded')

output = pd.DataFrame(final)
output.columns = ['u','shgc','leakage','lpd','people','elecCVRSME','chwCVRSME','heatCVRSME','elecNMBE','chwNMBE','heatNMBE']
output.to_csv('resultsOut.csv')
# plt.scatter(winHeight, dGain)
# plt.scatter(winHeight2, dGain)
# plt.xlabel("Window Height [m]")
# plt.ylabel("dGain [Btu/hr]")
# plt.legend(loc='lower right')
# plt.show()

mvRes.x
# def f2(u,shgc,leakage,lpd,people):
idf2.saveas('in.idf')




C:\EnergyPlusV22-1-0\energyplus.exe --weather C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1\USA_IL_Chicago.Midway.Intl.AP.725340_TMY3.epw --output-directory C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1 --idd C:\EnergyPlusV22-1-0\Energy+.idd --readvars C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1\in.idf

22.29369535441962
3177.5535667067934
42.42775175805358
[[1.0, 1.0, 1.0, 1.0, 1.0, 22.29369535441962, 3177.5535667067934, 42.42775175805358, 18.96364856438862, -2178.645266012353, 8.599448971835168]]

C:\EnergyPlusV22-1-0\energyplus.exe --weather C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1\USA_IL_Chicago.Midway.Intl.AP.725340_TMY3.epw --output-directory C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1 --idd C:\EnergyPlusV22-1-0\Energy+.idd --readvars C:\Users\amitc_crl\OneDrive\Documents\GitHub\IIT-PhD\CAE526\P1\in.idf

22.294397665124617
3178.2077613194583
42.31448517452473
[[1.0, 1.0, 1.0, 1.0, 1.0, 

In [ ]:
# output = pd.DataFrame(final)
# output.columns = ['u','shgc','leakage','lpd','people','elecCVRSME','chwCVRSME','heatCVRSME','elecCVRSME','chwCVRSME','heatCVRSME']
# output.to_csv('resultsOut.csv')